In [6]:
# Vamos a hacer un metamodelo para calcular el precio a mostrar en streamlit según las opciones que se vayan cogiendo

#### Preparativos

In [7]:
# Instalaciones
!pip install lightgbm

In [ ]:
# Importaciones

import pandas as pd
import numpy as np
import sklearn


from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import lightgbm as lgb
from lightgbm import LGBMRegressor

import pickle

In [29]:
print(sklearn.__version__)

1.5.1


In [10]:
print(lgb.__version__)

4.6.0


In [11]:
# Deslimitar/Limitar display Pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

#### Preparación Dataset

In [12]:
# Read Matches data

df0 = pd.read_csv(r'C:\DEV\TFM\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')

C:\Users\Diego\AppData\Local\Temp\ipykernel_12784\2799431303.py:3: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv(r'C:\DEV\TFM\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')


#### Preparación de un DF Mixto para CatBoost Básico

In [13]:
df_mix = df0.copy()

In [14]:
# Reconfigurar dtypes

df_mix.drop(columns=['ID_RESERVA'], inplace=True) # No usar, ID
df_mix.drop(columns=['ID_HOTEL'], inplace=True) # No usar, ID
df_mix['HOTEL'] = df_mix['HOTEL'].astype('category', errors='raise')
df_mix.drop(columns=['LLEGADA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['LLEGADA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['LLEGADA_MES'] = pd.to_numeric(df_mix['LLEGADA_MES'], errors='raise').astype('category')
df_mix['LLEGADA_DIAm'] = pd.to_numeric(df_mix['LLEGADA_DIAm'], errors='raise').astype('category')
df_mix['LLEGADA_DIAs'] = pd.to_numeric(df_mix['LLEGADA_DIAs'], errors='raise').astype('category')
df_mix['LLEGADA_AVANCE'] = pd.to_numeric(df_mix['LLEGADA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['SALIDA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['SALIDA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['SALIDA_MES'] = pd.to_numeric(df_mix['SALIDA_MES'], errors='raise').astype('category')
df_mix['SALIDA_DIAm'] = pd.to_numeric(df_mix['SALIDA_DIAm'], errors='raise').astype('category')
df_mix['SALIDA_DIAs'] = pd.to_numeric(df_mix['SALIDA_DIAs'], errors='raise').astype('category')
df_mix['SALIDA_AVANCE'] = pd.to_numeric(df_mix['SALIDA_AVANCE'], errors='raise').astype(float)
df_mix['NOCHES'] = pd.to_numeric(df_mix['NOCHES'], errors='raise').astype('Int64')
df_mix['DURACION_ESTANCIA'] = df_mix['DURACION_ESTANCIA'].astype('category', errors='raise')
df_mix['REGIMEN'] = df_mix['REGIMEN'].astype('category', errors='raise')
df_mix.drop(columns=['ID_TIPO'], inplace=True) # No usar, ID
df_mix['TIPO'] = df_mix['TIPO'].astype('category', errors='raise')
df_mix['USO'] = pd.to_numeric(df_mix['USO'], errors='raise').astype('Int64')
df_mix['PAX_NUM'] = pd.to_numeric(df_mix['PAX_NUM'], errors='raise').astype('Int64')
df_mix['PAX_CAT'] = df_mix['PAX_CAT'].astype('category', errors='raise')
df_mix['ADULTOS'] = pd.to_numeric(df_mix['ADULTOS'], errors='raise').astype('Int64')
df_mix['NENES'] = pd.to_numeric(df_mix['NENES'], errors='raise').astype('Int64')
df_mix['BEBES'] = pd.to_numeric(df_mix['BEBES'], errors='raise').astype('Int64')
df_mix.drop(columns=['ID_CLIENTE'], inplace=True) # No usar, ID
df_mix['TIPO_CLIENTE'] = pd.to_numeric(df_mix['TIPO_CLIENTE'], errors='raise').astype('category')
df_mix['CLIENTE'] = df_mix['CLIENTE'].astype('category', errors='raise')
df_mix['GRUPO'] = pd.to_numeric(df_mix['GRUPO'], errors='raise').astype('category')
df_mix.drop(columns=['ID_MONEDA'], inplace=True) # No usar, ID
df_mix['MONEDA'] = df_mix['MONEDA'].astype('category', errors='raise')
df_mix.drop(columns=['STATUS'], inplace=True) # No usar, redundante con variable objetivo
df_mix.drop(columns=['MOTIVO'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['CHECKIN'], inplace=True) # No usar, imposible ver el futuro
df_mix['SUPLETORIA'] = pd.to_numeric(df_mix['SUPLETORIA'], errors='raise').astype('Int64')
df_mix['CUNAS'] = pd.to_numeric(df_mix['CUNAS'], errors='raise').astype('Int64')
df_mix.drop(columns=['FECHA_TOMA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['FECHA_TOMA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['FECHA_TOMA_MES'] = pd.to_numeric(df_mix['FECHA_TOMA_MES'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAm'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAm'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAs'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAs'], errors='raise').astype('category')
df_mix['FECHA_TOMA_AVANCE'] = pd.to_numeric(df_mix['FECHA_TOMA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['FECHA_MOD'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix['LT_TOMA_LLEGADA'] = pd.to_numeric(df_mix['LT_TOMA_LLEGADA'], errors='raise').astype('Int64')
df_mix.drop(columns=['LT_TOMA_CANCELACION'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['ID_FIDELIDAD'], inplace=True) # No usar, ID
df_mix['FIDELIDAD'] = df_mix['FIDELIDAD'].astype('category', errors='raise')
df_mix.drop(columns=['VALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALFIJOS'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['COMERCIALIZADORA'] = pd.to_numeric(df_mix['COMERCIALIZADORA'], errors='raise').astype('category')
df_mix.drop(columns=['CMVALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMVALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMCVALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['VALOR_USD'] = pd.to_numeric(df_mix['VALOR_USD'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX'] = pd.to_numeric(df_mix['VALOR_USD_PAX'], errors='raise').astype(float)
df_mix['VALOR_USD_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_NOCHE'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_PAX_NOCHE'], errors='raise').astype(float)
df_mix.drop(columns=['AUTORIZO'], inplace=True) # No usar, de momento no muy claro
df_mix['GRATIS'] = pd.to_numeric(df_mix['GRATIS'], errors='raise').astype('category')
df_mix['PAIS'] = df_mix['PAIS'].astype('category', errors='raise')
df_mix['CONTINENTE'] = df_mix['CONTINENTE'].astype('category', errors='raise')
df_mix['SEGMENTO'] = df_mix['SEGMENTO'].astype('category', errors='raise')
df_mix['FUENTE_NEGOCIO'] = df_mix['FUENTE_NEGOCIO'].astype('category', errors='raise')
df_mix['CANCELADA'] = pd.to_numeric(df_mix['CANCELADA'], errors='raise').astype('category')

In [15]:
# Vamos a seleccionar las variables por intuición debido al poco tiempo y recursos disponibles,
# Pero lo óptimo sería usarlas todas o hacer un estudio SHAP
df_mix.columns

Index(['HOTEL', 'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs',
       'LLEGADA_AVANCE', 'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs',
       'SALIDA_AVANCE', 'NOCHES', 'DURACION_ESTANCIA', 'REGIMEN', 'TIPO',
       'USO', 'PAX_NUM', 'PAX_CAT', 'ADULTOS', 'NENES', 'BEBES',
       'TIPO_CLIENTE', 'CLIENTE', 'GRUPO', 'MONEDA', 'SUPLETORIA', 'CUNAS',
       'FECHA_TOMA_MES', 'FECHA_TOMA_DIAm', 'FECHA_TOMA_DIAs',
       'FECHA_TOMA_AVANCE', 'LT_TOMA_LLEGADA', 'FIDELIDAD', 'COMERCIALIZADORA',
       'VALOR_USD', 'VALOR_USD_PAX', 'VALOR_USD_NOCHE', 'VALOR_USD_PAX_NOCHE',
       'GRATIS', 'PAIS', 'CONTINENTE', 'SEGMENTO', 'FUENTE_NEGOCIO',
       'CANCELADA'],
      dtype='object')

In [16]:
df_precios = df_mix[[
    'HOTEL',
    'LLEGADA_MES',
    'LLEGADA_DIAm',
    'LLEGADA_DIAs',
    #'LLEGADA_AVANCE',
    #'SALIDA_MES',
    #'SALIDA_DIAm',
    #'SALIDA_DIAs',
    #'SALIDA_AVANCE',
    'NOCHES',
    #'DURACION_ESTANCIA',
    'REGIMEN',
    'TIPO',
    #'USO',
    'PAX_NUM',
    #'PAX_CAT',
    #'ADULTOS',
    #'NENES',
    #'BEBES',
    'TIPO_CLIENTE',
    #'CLIENTE',
    #'GRUPO',
    #'MONEDA',
    #'SUPLETORIA',
    #'CUNAS',
    #'FECHA_TOMA_MES',
    #'FECHA_TOMA_DIAm',
    #'FECHA_TOMA_DIAs',
    #'FECHA_TOMA_AVANCE',
    'LT_TOMA_LLEGADA',
    'FIDELIDAD',
    'COMERCIALIZADORA',
    'VALOR_USD', # -> Será la y
    #'VALOR_USD_PAX',
    #'VALOR_USD_NOCHE',
    #'VALOR_USD_PAX_NOCHE',
    #'GRATIS',
    #'PAIS',
    #'CONTINENTE',
    #'SEGMENTO',
    #'FUENTE_NEGOCIO',
    #'CANCELADA',
]]

df_precios.head()

,HOTEL,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,NOCHES,REGIMEN,TIPO,PAX_NUM,TIPO_CLIENTE,LT_TOMA_LLEGADA,FIDELIDAD,COMERCIALIZADORA,VALOR_USD
0,GrandPalladiumJamaica&LadyHamiltonResort,12,28,3,7,AllInclusive,LHAmbassadorSuiteBSOV,3,1,349,Ninguna,0,2576.00
1,GrandPalladiumJamaica&LadyHamiltonResort,1,22,6,7,AllInclusive,LHJuniorSuiteGV,3,1,193,Ninguna,0,1994.30
2,ComplejoRivieraMaya,11,6,1,8,AllInclusive,TRSJuniorSuitePS,2,1,203,Ninguna,0,2305.60
3,ComplejoRivieraMaya,11,6,1,17,AllInclusive,TRSJuniorSuitePS,2,1,257,Ninguna,0,4899.40
4,ComplejoRivieraMaya,11,25,6,14,AllInclusive,COLJuniorSuiteGV,1,1,281,Ninguna,0,2435.02


In [17]:
df_precios = df_precios.loc[df_precios['VALOR_USD'] != 0] # Para no confundir luego a los errores relativos

In [18]:
X = df_precios.drop(columns=['VALOR_USD'])

y = df_precios['VALOR_USD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Modelo Optimizado

In [19]:
# Preparación de la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [20]:
# Modelo LGBM Optimizado v3

Metamodelo_LGBM_Precios = LGBMRegressor(
    random_state=42,
)

param_grid_lgbm_regressor = {
    'n_estimators': [5000],
    'max_depth': [30],
    'learning_rate': [0.15],
    'num_leaves': [100],
}

grid_search_lgbm_regressor = GridSearchCV(
    estimator=Metamodelo_LGBM_Precios,
    param_grid=param_grid_lgbm_regressor,
    cv=kf,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    verbose=3,
    )

grid_search_lgbm_regressor.fit(X_train, y_train)
print(f"Best HyperParameters: {grid_search_lgbm_regressor.best_params_}")

# Asignar mejor estimador al modelo
Metamodelo_LGBM_Precios = grid_search_lgbm_regressor.best_estimator_

# Predicciones
y_test_pred = Metamodelo_LGBM_Precios.predict(X_test)

# Métricas
mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)


# Resultados sobre test
print("MSE :", mse)
print("RMSE :", mse**(1/2))
print("MAE :", mae)
print("Dado el gran rango de precios, es más coherente calcular los errores relativos")
print(f"MAPE : {(mape*100):.2f}%")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 569
[LightGBM] [Info] Number of data points in the train set: 909633, number of used features: 12
[LightGBM] [Info] Start training from score 1589.192576
Best HyperParameters: {'learning_rate': 0.15, 'max_depth': 30, 'n_estimators': 5000, 'num_leaves': 100}
MSE : 194544.31986260912
RMSE : 441.0717853848839
MAE : 213.00527274748907
Dado el gran rango de precios, es más coherente calcular los errores relativos
MAPE : 23.08%


In [21]:
df_precios_test = X_test.copy()
df_precios_test['VALOR_USD'] = y_test
df_precios_test['VALOR_USD_PRED'] = y_test_pred

In [22]:
df_precios_test.head(15)

,HOTEL,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,NOCHES,REGIMEN,TIPO,PAX_NUM,TIPO_CLIENTE,LT_TOMA_LLEGADA,FIDELIDAD,COMERCIALIZADORA,VALOR_USD,VALOR_USD_PRED
235517,ComplejoRivieraMaya,4,10,1,10,AllInclusive,TRSJSJacuzziTerrPS,2,1,102,Ninguna,0,3918.600000,4052.448217
964486,GrandPalladiumJamaica&LadyHamiltonResort,11,3,4,5,AllInclusive,LHJuniorSuiteBS,2,1,345,Ninguna,0,1462.500000,1591.161656
725499,PalladiumVallarta,8,7,1,3,AllInclusive,PVRDeluxeGardenView,2,1,170,Ninguna,0,566.059024,591.798677
436831,ComplejoPuntaCana,1,18,2,9,AllInclusive,TRSJuniorSuiteGV/PV,1,1,46,Ninguna,1,1333.800000,1310.150309
422671,ComplejoRivieraMaya,5,26,5,10,AllInclusive,WSJuniorSuiteGV,2,1,214,Ninguna,1,1424.600000,2030.739823
585322,ComplejoPuntaCana,2,18,6,7,AllInclusive,TRSJuniorSuiteGV/PV,2,1,129,Ninguna,1,2662.800000,2403.089104
548711,GrandPalladiumJamaica&LadyHamiltonResort,1,30,1,7,AllInclusive,JAMJuniorSuiteGV,2,1,22,Ninguna,1,1956.000000,1784.605457
580134,PalladiumVallarta,10,7,5,2,AllInclusive,PVRDeluxeOceanView,3,1,119,Ninguna,0,332.074627,408.546891
750743,ComplejoCostaMujeres,2,1,2,8,AllInclusive,CMUJuniorSuitePS,3,1,189,Ninguna,0,2611.040000,2568.070041
264629,TRSCapCana,2,20,1,2,AllInclusive,CAPJuniorSuiteMV,2,1,1,Ninguna,0,566.560000,796.493421


In [23]:
# Como demostrador para Streamlit, más que de sobra

#### Pickle

In [ ]:
with open("Metamodelo_LGBM_Precios.pkl", "wb") as f:
    pickle.dump(Metamodelo_LGBM_Precios, f)